# WeatherPy


In [12]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress

# Import API key
from api_keys import weather_api_key

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Output File (CSV)
output_data_file = "output_data/cities.csv"

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

## Generate Cities List

In [13]:
# List for holding lat_lngs and cities
lat_lngs = []
cities = []

# Create a set of random lat and lng combinations
lats = np.random.uniform(lat_range[0], lat_range[1], size=1500)
lngs = np.random.uniform(lng_range[0], lng_range[1], size=1500)
lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
len(cities)

591

### Perform API Calls
* Perform a weather check on each city using a series of successive API calls.
* Include a print log of each city as it'sbeing processed (with the city number and city name).


In [14]:
# Save config information
url = "http://api.openweathermap.org/data/2.5/weather?"

# Build query URL
query_url = url + "appid=" + weather_api_key 
query_url

'http://api.openweathermap.org/data/2.5/weather?appid=9d88a4e32284086ca01a89386c890bf8'

In [27]:
# set up lists to hold reponse info
city_names = []
#city_ids = []
count = 0
print(f"Beginning Data Retrieval: \n")
print(f"------------------------\n" )
# Loop through the list of cities and perform a request for data on each
for city in cities[:10]:
    city_url = query_url + "&q=" + city
    count += 1 
# Get weather data
    weather_response = requests.get(city_url)
    weather_json = weather_response.json()

# Try to grab the city name and id if they are available in the weather API
    try:
        
        city_name = weather_json['name']
        #city_id = weather_json['id']
        
        city_names.append(city_name)
        #city_ids.append(city_id)

        print(f"Processing Record {count} | {city_name}")
        
# Handle exceptions for a city that is not available in the weather API
    except:
        # Append null values
        print(f"City not found. Skipping...")
        pass


      
#STILL NEED TO GET COUNTER TO NOT COUNT THE SKIPS   

Beginning Data Retrieval: 

------------------------

Processing Record 1 | Albany
City not found. Skipping...
Processing Record 3 | Atar
Processing Record 4 | Manosque
Processing Record 5 | Bredasdorp
Processing Record 6 | Rikitea
Processing Record 7 | Murom
Processing Record 8 | East London
Processing Record 9 | Mataura
Processing Record 10 | Port Hardy


### Convert Raw Data to DataFrame
* Export the city data into a .csv.
* Display the DataFrame

In [31]:
#name_city = weather_json['name']
longitude = weather_json["lon"]
latitude = weather_json['lat']
max_temp = weather_json['temp_max']
humidity = weather_json['humidity']
cloudiness = weather_json['all']
wind_speed = weather_json['speed']
country = weather_json['country']
date = weather_json['dt']


# create a data frame from cities, lat, and temp
weather_dict = {
    "City": city_name,
    "Latitude": lats,
    "Longitude": lngs,
    "Max Temp": max_temp,
    "Humidity": humidity,
    "Cloudiness": cloudiness,
    "Wind Speed": wind_speed,
    "Country": country,
    "Date": date
}
weather_data = pd.DataFrame(weather_dict)
weather_data

KeyError: 'lon'

## Inspect the data and remove the cities where the humidity > 100%.
----
Skip this step if there are no cities that have humidity > 100%. 

In [ ]:
#  Get the indices of cities that have humidity over 100%.


In [ ]:
# Make a new DataFrame equal to the city data to drop all humidity outliers by index.
# Passing "inplace=False" will make a copy of the city_data DataFrame, which we call "clean_city_data".


In [ ]:
# Extract relevant fields from the data frame


# Export the City_Data into a csv


## Plotting the Data
* Use proper labeling of the plots using plot titles (including date of analysis) and axes labels.
* Save the plotted figures as .pngs.

## Latitude vs. Temperature Plot

## Latitude vs. Humidity Plot

## Latitude vs. Cloudiness Plot

## Latitude vs. Wind Speed Plot

## Linear Regression

In [ ]:
# OPTIONAL: Create a function to create Linear Regression plots

In [ ]:
# Create Northern and Southern Hemisphere DataFrames

####  Northern Hemisphere - Max Temp vs. Latitude Linear Regression

####  Southern Hemisphere - Max Temp vs. Latitude Linear Regression

####  Northern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression

####  Southern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression